In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..')

from collections import defaultdict
from datetime import datetime
from itertools import combinations
import json
import os
from multiprocessing import Pool
from collections import Counter

from tqdm.auto import tqdm

from paradeller.helper import DATE_FMT, load_archive
from paradeller.dataprep import (
    load_and_prep,
    sort_ids_by_popularity,
    restructure_data,
    create_adj_list_by_id,
    create_adj_list_by_word,
    filter_out_oddballs_recursive,
)   
from paradeller.analysis import (
    consolidate_stanzas,
    find_matches,
    find_matches_for_start_pairs,
    get_num_combos
)
from paradeller.postprocess import (
    stanza_sorter_maker, print_stanzas, print_poems, print_stanza
)

In [3]:
def get_tweet(i):
    """Find tweet with given id"""
    try:
        return next(x for x in data if x["id"] == i)
    except StopIteration as e:
        print("Error: No tweet with that ID")
        return None

showlen = lambda data: print(f"{len(data):,}")

In [5]:
data, duplicates, adj_list_words, adj_list_ids = load_and_prep(use_pickle=True, update_pickle=False)


Loading processed data from pickle...
--------------------------------------------------
DONE

data            type: <class 'list'>	len: 270,885
duplicates      type: <class 'dict'>	len: 5,365
adj_list_words  type: <class 'dict'>	len: 22,505
adj_list_ids    type: <class 'dict'>	len: 270,885


In [4]:
data, duplicates, adj_list_words, adj_list_ids = load_and_prep(use_pickle=False, update_pickle=True)


Loading raw data from archive.json...
Length: 394,026

Cleaning up data...
> Remove too short



Length: 361,234
> Remove duplicate phrases



Length: 349,960
> Recursively remove oddballs



69,794 tweets removed. Running again.



7,780 tweets removed. Running again.


...


14 tweets removed. Running again.



13 tweets removed. Running again.



4 tweets removed. Running again.



Nothing removed. Done filtering.
Length: 270,885

Creating adjacency lists...




Restructing duplicates...

Saving new data to pickle...
--------------------------------------------------
DONE

data            type: <class 'list'>	len: 270,885
duplicates      type: <class 'dict'>	len: 5,365
adj_list_words  type: <class 'dict'>	len: 22,505
adj_list_ids    type: <class 'dict'>	len: 270,885


## More filtering??

### Word Count Brillaince

In [ ]:
before = len(data)
# before = 254123
before

In [ ]:
def can_be_completed(line, overall_wc):
    counts = Counter(adj_list_ids[line])
    return all([
        overall_wc[word] - counts[word] >= (counts[word] * 2)
        for word in counts
    ])

In [ ]:
def recusively_remove_losers(data, adj_list_ids):
    showlen(data)
    
    # get overall word count
    overall_wc = Counter()
    for id_, words in adj_list_ids.items():
        overall_wc.update(words)
    
    # try removing some ids
    ids = list(adj_list_ids.keys())
    rm_ids = set([i for i in ids if not can_be_completed(i, overall_wc)])
    
    if len(rm_ids) > 0:
        print(f"Removing {len(rm_ids)} ids")
        data = [x for x in data if x["id"] not in rm_ids]
        adj_list_ids = create_adj_list_by_id(data)
        return recusively_remove_losers(data, adj_list_ids)
    
    print("All done filtering! Updating adj lists")
    adj_list_words, adj_list_ids = restructure_data(data)
    return data, adj_list_words, adj_list_ids

In [ ]:
data, adj_list_words, adj_list_ids  = recusively_remove_losers(data, adj_list_ids)

In [ ]:
data = filter_out_oddballs_recursive(data)

In [ ]:
after = len(data)
after

In [ ]:
before - after

In [ ]:
b_num = get_num_combos(before, 2)
print(f"{b_num:,}")

In [ ]:
a_num = get_num_combos(after, 2)
print(f"{a_num:,}")

#### Search least popular

In [ ]:
from statistics import mean

In [ ]:
sorted_ids = sort_ids_by_popularity(adj_list_ids, adj_list_words)

some_ids = sorted_ids[:100]
pairs = list(combinations(some_ids, 2))

len(pairs)

In [ ]:
ids = list(adj_list_ids.keys())

In [ ]:
least_pop_ids = sorted_ids[-3000:]

In [ ]:
pairs = list(combinations(least_pop_ids, 2))

In [ ]:
len(pairs)

In [ ]:
# ids = least_pop_ids
# pairs = combinations(ids, 2)

# num_combos = get_num_combos(len(ids), 2)
# print(f"{num_combos:,}")

In [ ]:
l = least_pop_ids[-1]

In [ ]:
len(ids)

In [ ]:
pairs = [(l, i) for i in ids]

In [ ]:
len(pairs)

In [ ]:
pairs[0]

In [ ]:
# get overall word count
overall_wc = Counter()
for id_, words in adj_list_ids.items():
    overall_wc.update(words)

In [ ]:
def valid_pair(id1, id2):
    stanza_words = adj_list_ids[id1] + adj_list_ids[id2]
    counts = Counter(stanza_words)
    return all([
        overall_wc[word] - counts[word] >= (counts[word] * 2)
        for word in counts
    ])

In [ ]:
shorter = [p for p in tqdm(pairs) if valid_pair(*p)]

In [ ]:
len(pairs) - len(shorter)

In [ ]:
id1, id2 = pairs[0]

In [ ]:
id1

In [ ]:
get_tweet(id1)

In [ ]:
stanza_words = adj_list_ids[id1] + adj_list_ids[id2]

In [ ]:
counts = Counter(stanza_words)

In [ ]:
counts

In [ ]:
stanza_words

In [ ]:
overall_wc['15']

In [ ]:
[
    overall_wc[word] - counts[word]
    for word in counts
]

In [ ]:
for i in adj_list_words['liat']:
    print(adj_list_ids[i])

In [ ]:
[
    overall_wc[word] - counts[word] >= (counts[word] * 2)
    for word in counts
]

In [ ]:
253690 - len(shorter)

In [ ]:
for id1, id2 in pairs:
    
    counts = Counter(stanza_words)

In [ ]:
# sorted_ids = sort_ids_by_popularity(adj_list_ids, adj_list_words)

some_ids = sorted_ids[:100]
pairs = list(combinations(some_ids, 2))

len(pairs)

In [ ]:
def find_matches_for_pair(p):
    """Helper function to find initial stanzas, given a pair of lines"""
    return find_matches(p[0], p[1], adj_list_ids, adj_list_words)

In [ ]:
with Pool(os.cpu_count()) as pool:
    res = list(tqdm(pool.imap(find_matches_for_pair, pairs), total=len(pairs)))

valid_stanzas = [x for x in list(zip(pairs, res)) if x[1]]
stanzas = consolidate_stanzas(valid_stanzas)

print(f"Found {len(stanzas)} results.")

In [ ]:
d = defaultdict(list)
for pair in tqdm(pairs, total=num_combos):
    w1 = adj_list_ids[pair[0]]
    w2 = adj_list_ids[pair[1]]
    words = tuple(sorted(w1 + w2))
    d[words].append(pair)

In [ ]:
found = {k:v for k, v in d.items() if len(v) > 1}
len(found)

In [ ]:
valid_stanzas = []
for matching_pairs in tqdm(found.values()):
    combos = list(combinations(matching_pairs, 2))
    stanzas = combos
    stanzas = [c[0] + c[1] for c in combos]
    valid_stanzas.extend(stanzas)
    
print("valid_stanzas:", len(valid_stanzas))

In [ ]:
keep_ids = set().union(*valid_stanzas)

In [ ]:
len(keep_ids)

In [ ]:
# remove_ids = set(least_pop_ids) - keep_ids

In [ ]:
print_stanzas(valid_stanzas, data, n=3)

### Filter down found stanzas

Using word counts

In [ ]:
overall_wc = Counter()
for id_, words in adj_list_ids.items():
    overall_wc.update(words)

In [ ]:
def can_be_completed(stanza):
    a, b, _, _ = stanza
    
    counts = Counter()
    counts.update(adj_list_ids[a] + adj_list_ids[b])
    
    return all([
        overall_wc[word] - (counts[word] * 2) >= counts[word]
        for word in counts
    ])

In [ ]:
shorter = [x for x in valid_stanzas if can_be_completed(x)]

In [ ]:
len(valid_stanzas)

In [ ]:
elim = list(set(valid_stanzas) - set(shorter))

In [ ]:
print_stanzas(elim, data)

In [ ]:
overall_wc['toon']

In [ ]:
40 - 15 - 15

In [ ]:
word_counts['toon'] > 15 * 2

In [ ]:
15 * 2

In [ ]:
g = Counter()

In [ ]:
g.update(adj_list_ids[c]+adj_list_ids[d])

In [ ]:
g

In [ ]:
stanza

In [ ]:
print_stanza(stanza, data)

#### count words

## Alt Approach

In [ ]:
ids = list(adj_list_ids.keys())
len(ids)

In [ ]:
pairs = combinations(ids, 2)

num_combos = get_num_combos(len(ids), 2)
print(f"{num_combos:,}")

In [ ]:
d = defaultdict(list)
for pair in tqdm(pairs, total=num_combos):
    w1 = adj_list_ids[pair[0]]
    w2 = adj_list_ids[pair[1]]
    words = tuple(sorted(w1 + w2))
    d[words].append(pair)

### Find Stanzas

#### Old

In [ ]:
def find_matches_for_pair(p):
    """Helper function to find initial stanzas, given a pair of lines"""
    return find_matches(p[0], p[1], adj_list_ids, adj_list_words)

In [ ]:
with Pool(os.cpu_count()) as pool:
    res = list(tqdm(pool.imap(find_matches_for_pair, pairs), total=len(pairs)))

In [ ]:
valid_stanzas = [x for x in list(zip(pairs, res)) if x[1]]
stanzas = consolidate_stanzas(valid_stanzas)
print(f"Found {len(stanzas)} results.")

In [ ]:
old_stanzas = stanzas

#### New

In [ ]:
ids = list(adj_list_ids.keys())
# pairs = list(combinations(ids, 2))

# len(pairs)

In [ ]:
d = defaultdict(list)
for pair in tqdm(pairs):
    w1 = adj_list_ids[pair[0]]
    w2 = adj_list_ids[pair[1]]
    words = tuple(sorted(w1 + w2))
    d[words].append(pair)

In [ ]:
found = {k:v for k, v in d.items() if len(v) > 1}
len(found)

In [ ]:
valid_stanzas = []
for matching_pairs in tqdm(found.values()):
    combos = list(combinations(matching_pairs, 2))
    stanzas = [c[0] + c[1] for c in combos]
    valid_stanzas.extend(stanzas)
    
print("valid_stanzas:", len(valid_stanzas))

In [ ]:
new_stanzas = valid_stanzas

In [ ]:
len(old_stanzas)

In [ ]:
len(new_stanzas)

In [ ]:
print_stanzas(new_stanzas, data)

In [ ]:
old_stanzas[0]

In [ ]:
print_stanza(old_stanzas[0], data)

In [ ]:
print_stanzas(old_stanzas, data)

In [ ]:
for k in list(found.keys())[:5]:
    print(k, ":")
    print("  ", found[k])

In [ ]:
print_stanzas(valid_stanzas, data)